# RAG chat over web source
- adopted from https://python.langchain.com/docs/use_cases/question_answering/ 

In [1]:
import os
import openai
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
# from langchain_core.runnables import RunnablePassthrough
from langchain.schema.runnable  import RunnablePassthrough
from ssl_workaround import no_ssl_verification
from dotenv import load_dotenv, find_dotenv

# 1. Init

In [2]:
def init():
    _ = load_dotenv(find_dotenv()) # read local .env file
    openai.api_key = os.environ['OPENAI_API_KEY']
    openai.api_base = os.environ['OPENAI_API_BASE']
    openai.api_type= os.environ['OPENAI_API_TYPE']
    openai.api_version = os.environ['OPENAI_API_VERSION']
    print(f'Openai secrets loaded, models: {os.environ["OPENAI_DEPLOYMENT_ID_LLM"]}, {os.environ["OPENAI_DEPLOYMENT_ID_EMBED"]}')
init()

Openai secrets loaded, models: gpt-35-turbo, text-embedding-ada-002


## 2. Load content from Web

In [3]:
# loads URL content into a document - one document per URL
# only extracts the content of the post-content, post-title, and post-header classes
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    },
)
docs = loader.load()

In [4]:
# test doc loading
print(f'Nuber of docs loaded: {len(docs)}')
print(f'First doc lenght: {len(docs[0].page_content)}')
print(f'Sample: {docs[0].page_content[:500]}')

Nuber of docs loaded: 1
First doc lenght: 42824
Sample: 

      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


# 3. Split
- LLM struggle to find info in very long context, we should split it in reasonable long documents
- its good to have overlap between spit documents in order not to loose context from ending / begginging chunk parts

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [6]:
# test splitter
print(f'Number of documents created: {len(all_splits)}')
print(f'First document length: {len(all_splits[0].page_content)}')
print(f'Metadata sample: {all_splits[0].metadata}')

Number of documents created: 66
First document length: 969
Metadata sample: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 8}


In [7]:
# TODO - optimize with https://python.langchain.com/docs/use_cases/question_answering/#go-deeper-1

# 4. Vector store
- embed and store all documents in vector store 

In [7]:
embedding=AzureOpenAIEmbeddings(azure_deployment=os.environ['OPENAI_DEPLOYMENT_ID_EMBED'])

In [8]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding)

# 5. Retrieve

In [9]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [10]:
# retriever test
retrieved_docs = retriever.get_relevant_documents(
    "What are the approaches to Task Decomposition?"
)
print(f'---Number of documents retrieved: {len(retrieved_docs)}')
print(f'---Doc 0 length: {len(retrieved_docs[0].page_content)}')
print(f'---Doc 0: {retrieved_docs[0].page_content}')
print(f'---Doc 1 length: {len(retrieved_docs[1].page_content)}')
print(f'---Doc 1: {retrieved_docs[1].page_content}')

---Number of documents retrieved: 6
---Doc 0 length: 644
---Doc 0: Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
---Doc 1 length: 606
---Doc 1: Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a

# 6. Generate

In [11]:
llm = AzureChatOpenAI(azure_deployment=os.environ['OPENAI_DEPLOYMENT_ID_LLM'])

In [12]:
#get prompt template
prompt = hub.pull("rlm/rag-prompt")

In [15]:
# test prompt
print(
    prompt.invoke(
        {"context": "filler context", "question": "filler question"}
    ).to_string()
)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


## create chain
- LangChain Expression Language (LCEL)

In [13]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
with no_ssl_verification():
    for chunk in rag_chain.stream("What is Task Decomposition?"):
        print(chunk, end="", flush=True)

Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This can be done through various methods such as using prompting techniques like Chain of Thought or Tree of Thoughts, task-specific instructions, or human inputs. LLM works as the brain in the task planning stage, parsing user requests into multiple tasks with four attributes: task type, ID, dependencies, and arguments.

In [19]:
# with no_ssl_verification():
#     rag_chain.invoke("What is Task Decomposition?")

## customize the prompt

In [21]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "Cheers!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)

with no_ssl_verification():
    rag_chain.invoke("What is Task Decomposition?")

# 7. Add sources

In [22]:
from operator import itemgetter

from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    {
        "context": lambda input: format_docs(input["documents"]),
        "question": itemgetter("question"),
    }
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)
rag_chain_with_source = RunnableParallel(
    {"documents": retriever, "question": RunnablePassthrough()}
) | {
    "documents": lambda input: [doc.metadata for doc in input["documents"]],
    "answer": rag_chain_from_docs,
}

rag_chain_with_source.invoke("What is Task Decomposition")

Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-09-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 5 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


{'documents': [{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
   'start_index': 1585},
  {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
   'start_index': 2192},
  {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
   'start_index': 17804},
  {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
   'start_index': 17414},
  {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
   'start_index': 29630},
  {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
   'start_index': 19373}],
 'answer': 'Task decomposition is the process of breaking down a complex task into smaller, more manageable steps. This can be done through techniques such as Chain of Thought or Tree of Thoughts, and can be facilitated by LLM, task-specific instructions, or human inputs. Cheers!'}

# 8. Add Memory
- for statefull apps
- this can be done by refolmulating the latest question given the history
- add a call to LLM to refolmulate

In [24]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

condense_q_system_prompt = """Given a chat history and the latest user question \
which might reference the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
condense_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
condense_q_chain = condense_q_prompt | llm | StrOutputParser()

In [25]:
condense_q_chain

ChatPromptTemplate(input_variables=['chat_history', 'question'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user question which might reference the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is.')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))])
| AzureChatOpenAI(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, openai_api_key='2

In [26]:
# test chat chain
from langchain_core.messages import AIMessage, HumanMessage

condense_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "What is meant by large",
    }
)

'The term "large" in reference to language models generally means that the model has been trained on a very large corpus of text data, often on the scale of billions or trillions of words.'

In [31]:
# QA chain
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def condense_question(input: dict):
    if input.get("chat_history"):
        return condense_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(context=condense_question | retriever | format_docs)
    | qa_prompt
    | llm
)

In [32]:
chat_history = []

question = "What is Task Decomposition?"
with no_ssl_verification():
    ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])


In [34]:
second_question = "What are common ways of doing it?"

with no_ssl_verification():
    rag_chain.invoke({"question": second_question, "chat_history": chat_history})

chat_history


Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-09-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 10 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


RateLimitError: Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-09-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 5 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.

this approach has issue with Azure Open AI call rate limit

# Approach 2 - Using conversational chain

In [ ]:
# !pip install tiktoken chromadb

In [ ]:
# from langchain.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
# data = loader.load()

In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# all_splits = text_splitter.split_documents(data)

# from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import Chroma

# vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [ ]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)

In [ ]:
from langchain.chains import ConversationalRetrievalChain
# from langchain.chat_models import ChatOpenAI

# llm = ChatOpenAI()
# retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
qa("How do agents use Task decomposition?")

Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-07-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 10 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


RateLimitError: Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-07-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 6 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.

this approach has issue with Azure Open AI call rate limit

In [ ]:
qa("What are the various ways to implement memory to support it?")

# Clean up

In [ ]:
# cleanup
vectorstore.delete_collection()